## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [141]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.enable_eager_execution()

### Collect Data

In [143]:
from google.colab import drive
#drive.mount('/gdrive')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
dataPrices = pd.read_csv('/content/drive/My Drive/GreatLearning/myprojects/Residency6/prices.csv')

### Check all columns in the dataset

In [146]:
dataPrices.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
dataPrices.drop(['date','symbol'],axis=1, inplace = True)

In [148]:
dataPrices.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
dataPrices1000 = dataPrices.head(1000)

In [0]:
dataPrices1000.volume = dataPrices1000.volume/1000000

In [151]:
dataPrices1000.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [152]:
dataPrices1000.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
X = dataPrices1000.drop('volume',axis = 1)
Y = dataPrices1000.volume

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x = np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.zeros(shape=(4,2))
b1 = tf.zeros(shape=(2))

#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.zeros(shape=(2,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(X_train, Y_train, w1, b1,w2,b2, learning_rate=0.01):
    
    with tf.GradientTape() as t: #tf.GradientTape records all operations in forward pass to apply gradient later

      t.watch([w1,b1,w2,b2])

      predicted, predicted2   = prediction(train_x, w1, b1,w2,b2)
      curr_loss = loss(predicted, Y_train)

    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(curr_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    return w1, b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [161]:
for i in range(100):
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1, w2, b2)
    print("Loss at step {:d}: {:.3f}".format(i, loss(prediction(train_x, w1, b1,w2,b2), train_y)))
    print("Loss at step {:d}: {:.3f}".format(i + 5, loss(prediction(train_x, w1, b1,w2,b2), train_y)))

Loss at step 0: 246.707
Loss at step 5: 246.707
Loss at step 1: 245.613
Loss at step 6: 245.613
Loss at step 2: 244.581
Loss at step 7: 244.581
Loss at step 3: 243.608
Loss at step 8: 243.608
Loss at step 4: 242.692
Loss at step 9: 242.692
Loss at step 5: 241.829
Loss at step 10: 241.829
Loss at step 6: 241.016
Loss at step 11: 241.016
Loss at step 7: 240.251
Loss at step 12: 240.251
Loss at step 8: 239.531
Loss at step 13: 239.531
Loss at step 9: 238.853
Loss at step 14: 238.853
Loss at step 10: 238.215
Loss at step 15: 238.215
Loss at step 11: 237.616
Loss at step 16: 237.616
Loss at step 12: 237.051
Loss at step 17: 237.051
Loss at step 13: 236.521
Loss at step 18: 236.521
Loss at step 14: 236.022
Loss at step 19: 236.022
Loss at step 15: 235.554
Loss at step 20: 235.554
Loss at step 16: 235.114
Loss at step 21: 235.114
Loss at step 17: 234.702
Loss at step 22: 234.702
Loss at step 18: 234.315
Loss at step 23: 234.315
Loss at step 19: 233.953
Loss at step 24: 233.953
Loss at step 20

### Get the shapes and values of W and b

In [162]:
print('Shape of w1:')
print(w1.shape)
print('Shape of w2:')
print(w2.shape)
print('Shape of b1:')
print(b1.shape)
print('Shape of b2:')
print(b2.shape)

Shape of w1:
(4, 2)
Shape of w2:
(2, 1)
Shape of b1:
(2,)
Shape of b2:
(1,)


In [163]:
print('Value of w1:')
print(w1)
print('Value of w2:')
print(w2)
print('Value of b1:')
print(b1)
print('Value of b2:')
print(b2)

Value of w1:
tf.Tensor(
[[0.26847374 0.26847374]
 [0.2694031  0.2694031 ]
 [0.2657035  0.2657035 ]
 [0.27174404 0.27174404]], shape=(4, 2), dtype=float32)
Value of w2:
tf.Tensor(
[[1.7460142]
 [1.7460142]], shape=(2, 1), dtype=float32)
Value of b1:
tf.Tensor([0.5376929 0.5376929], shape=(2,), dtype=float32)
Value of b2:
tf.Tensor([2.9886782], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [164]:
y_pred,_ = prediction(test_x,w1,b1,w2,b2)
y_pred

<tf.Tensor: id=10124220, shape=(300, 1), dtype=float32, numpy=
array([[5.592392 ],
       [5.592392 ],
       [5.59239  ],
       [5.592363 ],
       [5.592394 ],
       [5.5923853],
       [5.59239  ],
       [5.59239  ],
       [5.59239  ],
       [5.5923886],
       [5.592391 ],
       [5.5923924],
       [5.5923777],
       [5.592394 ],
       [5.592392 ],
       [5.592389 ],
       [5.5923724],
       [5.592393 ],
       [5.592392 ],
       [5.592383 ],
       [5.592388 ],
       [5.592392 ],
       [5.592388 ],
       [5.5923853],
       [5.59239  ],
       [5.5923915],
       [5.592394 ],
       [5.592389 ],
       [5.5923924],
       [5.592392 ],
       [5.592352 ],
       [5.59239  ],
       [5.592392 ],
       [5.5923934],
       [5.59239  ],
       [5.59239  ],
       [5.5923767],
       [5.592393 ],
       [5.59239  ],
       [5.5923877],
       [5.5923834],
       [5.592388 ],
       [5.592391 ],
       [5.5923934],
       [5.592391 ],
       [5.5923753],
       [5.592392 

In [172]:
y_pred.shape

TensorShape([300, 1])

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [165]:
dataIris = pd.read_csv('/content/drive/My Drive/GreatLearning/myprojects/Residency6/11_Iris.csv')
dataIris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [166]:
dataIris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [167]:
dataIris.Species.value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [0]:
dataIrisEncoded = pd.get_dummies(dataIris.Species)

In [169]:
dataIrisEncoded.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [170]:
dataIris.shape

(150, 6)

In [171]:
dataIris.columns

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species'],
      dtype='object')

### Splitting the data into feature set and target set

In [0]:
X = dataIris.drop(['Species','Id'],axis = 1)
Y = dataIrisEncoded

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Model
from keras.models import load_model

In [0]:
# Initialize Sequential model
model = Sequential()

In [0]:
model.add(Dense(4, activation='relu', input_dim=4))

In [0]:
#Output layer
model.add(Dense(3, activation='softmax', name='Output'))

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [180]:
model.fit(train_x, train_y, epochs=100)

Train on 105 samples
Epoch 1/100
105/105 [==============================] - 0s 472us/sample - loss: 2.2693 - accuracy: 0.3143
Epoch 2/100
105/105 [==============================] - 0s 89us/sample - loss: 1.4574 - accuracy: 0.2952
Epoch 3/100
105/105 [==============================] - 0s 95us/sample - loss: 1.1575 - accuracy: 0.1143
Epoch 4/100
105/105 [==============================] - 0s 94us/sample - loss: 1.0660 - accuracy: 0.2571
Epoch 5/100
105/105 [==============================] - 0s 98us/sample - loss: 1.0231 - accuracy: 0.2381
Epoch 6/100
105/105 [==============================] - 0s 94us/sample - loss: 0.9977 - accuracy: 0.2381
Epoch 7/100
105/105 [==============================] - 0s 93us/sample - loss: 0.9767 - accuracy: 0.2667
Epoch 8/100
105/105 [==============================] - 0s 84us/sample - loss: 0.9366 - accuracy: 0.3429
Epoch 9/100
105/105 [==============================] - 0s 74us/sample - loss: 0.8994 - accuracy: 0.4286
Epoch 10/100
105/105 [====================

### Model Prediction

In [181]:
model.predict(test_x)

array([[9.7217834e-01, 2.2908704e-02, 4.9129515e-03],
       [9.2573637e-01, 5.9811369e-02, 1.4452255e-02],
       [8.4880935e-03, 4.1943994e-01, 5.7207197e-01],
       [9.1674870e-01, 6.7806438e-02, 1.5444912e-02],
       [9.0172881e-01, 7.7884071e-02, 2.0387176e-02],
       [2.4698791e-03, 4.0151468e-01, 5.9601539e-01],
       [8.8122821e-01, 9.6075647e-02, 2.2696048e-02],
       [1.2773382e-03, 3.2103828e-01, 6.7768443e-01],
       [4.5440355e-03, 3.9097601e-01, 6.0447997e-01],
       [9.3219590e-01, 5.5464331e-02, 1.2339764e-02],
       [9.7785521e-01, 1.8721178e-02, 3.4235890e-03],
       [9.6287596e-01, 3.0290017e-02, 6.8340208e-03],
       [9.6813411e-01, 2.6815277e-02, 5.0505535e-03],
       [9.8148900e-01, 1.5633829e-02, 2.8770675e-03],
       [6.4840145e-02, 5.9361619e-01, 3.4154359e-01],
       [1.5322893e-02, 5.7277679e-01, 4.1190025e-01],
       [7.8782511e-01, 1.6813940e-01, 4.4035479e-02],
       [3.9153922e-02, 5.7289702e-01, 3.8794905e-01],
       [1.2194569e-02, 4.799

In [182]:
print('Model Evaluation on test data: ')
results = model.evaluate(test_x, test_y, verbose=0)
print('Test loss, Test Accuracy : ', results)

Model Evaluation on test data: 
('Test loss, Test Accuracy : ', [0.39128103653589885, 0.9111111])


### Save the Model

In [183]:
model.save('/content/drive/My Drive/GreatLearning/myprojects/Residency6/Lab_R6_Internal_model.h5')
print('Saved model in given path in google drive.....')

Saved model in given path in google drive.....


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?